In [1]:
from __future__ import division
import numpy as np
import glob
import numexpr

In [2]:
electronvolt = 1.0e-9;
kiloelectronvolt = 1.e+3*electronvolt;
megaelectronvolt = 1.e+6*electronvolt; 
gigaelectronvolt = 1.e+9*electronvolt;
teraelectronvolt = 1.e+12*electronvolt;
petaelectronvolt = 1.e+15*electronvolt;
MeV = megaelectronvolt;
eV = electronvolt;
keV = kiloelectronvolt;
GeV = gigaelectronvolt;
TeV = teraelectronvolt;
PeV = petaelectronvolt;

class baseEnum(int):
    name = None
    values = {}
    def __repr__(self):
        return self.name

class metaEnum(type):
    def __new__(cls, classname, bases, classdict):
        newdict = {"values":{}}
        for k in classdict.keys():
            if not (k.startswith('_') or k == 'name' or k == 'values'):
                val = classdict[k]
                member = baseEnum(val)
                member.name = k
                newdict['values'][val] = member
                newdict[k] = member
              
        # Tell each member about the values in the enum
        for k in newdict['values'].keys():
            newdict['values'][k].values = newdict['values']
        # Return a new class with the "values" attribute filled
        return type.__new__(cls, classname, bases, newdict)

class enum(baseEnum, metaclass=metaEnum):
    """This class mimicks the interface of boost-python-wrapped enums.
      
Inherit from this class to construct enumerated types that can
be passed to the I3Datatype, e.g.:

        class DummyEnummy(tableio.enum):
                Foo = 0
               Bar = 1
                Baz = 2

       desc = tableio.I3TableRowDescription()
       desc.add_field('dummy', tableio.I3Datatype(DummyEnummy), '', '')
"""
class ParticleType(enum):
    PPlus       =   14
    He4Nucleus  =  402
    N14Nucleus  = 1407
    O16Nucleus  = 1608
    Al27Nucleus = 2713
    Fe56Nucleus = 5626
    NuE         =   66
    NuEBar      =   67
    NuMu        =   68
    NuMuBar     =   69
    NuTau       =  133
    NuTauBar    =  134

class PDGCode(enum):
    PPlus       =       2212
    He4Nucleus  = 1000020040
    N14Nucleus  = 1000070140
    O16Nucleus  = 1000080160
    Al27Nucleus = 1000130270
    Fe56Nucleus = 1000260560
    NuE         =         12
    NuEBar      =        -12
    NuMu        =         14
    NuMuBar     =        -14
    NuTau       =         16
    NuTauBar    =        -16
PDGCode.from_corsika = classmethod(lambda cls, i: getattr(cls, ParticleType.values[i].name))
ParticleType.from_pdg = classmethod(lambda cls, i: getattr(cls, PDGCode.values[i].name))
        
def build_lookup(mapping, var='ptype', default='ptype'):
    """
    Build an expression equivalent to a lookup table
    """
    if len(mapping) > 0:
        return 'where(%s==%s, %s, %s)' % (var, mapping[0][0], mapping[0][1], build_lookup(mapping[1:], var, default))
    else:
        return str(default)
class CompiledFlux(object):
    """
    An efficient pre-compiled form of a multi-component flux. For single-element evalutions
    this is ~2 times faster than switching on the primary type with an if statement; for 1e5
    samples it is 2000 times faster than operating on masked slices for each primary type.
    """
    pdg_to_corsika = numexpr.NumExpr(build_lookup([(int(PDGCode.from_corsika(v)), v) for v in ParticleType.values.keys()]))
    def __init__(self, expr):
        self.expr = numexpr.NumExpr(expr)
        # by default, assume PDG codes
        self._translator = CompiledFlux.pdg_to_corsika
       
    def to_PDG(self):
        """
        Convert to a form that takes PDG codes rather than CORSIKA codes.
        """
        new = copy.copy(self)
        new._translator = CompiledFlux.pdg_to_corsika
        return new
       
    def __call__(self, E, ptype):
        """
        :param E: particle energy in GeV
        :param ptype: particle type code
        :type ptype: int
        """
        if self._translator:
            ptype = self._translator(ptype)
            return self.expr(E, ptype)
       
    @staticmethod
    def build_lookup(mapping, var='ptype', default=0.):
        """
        Build an expression equivalent to a lookup table
        """
        # force mapping to be a list if it wasn't already
        mapping=list(mapping)
        if len(mapping) > 0:
            return 'where(%s==%s, %s, %s)' % (var, mapping[0][0], mapping[0][1], build_lookup(mapping[1:], var, default))
        else:
            return str(default)

class GaisserHillas(CompiledFlux):
    
    ptypes = [getattr(ParticleType, p) for p in ('PPlus', 'He4Nucleus', 'N14Nucleus', 'Al27Nucleus', 'Fe56Nucleus')]
    def get_expression(self, flux, gamma, rigidity):
        z = "where(ptype > 100, ptype%100, 1)"
        return "%(flux)s*E**(-%(gamma)s)*exp(-E/(%(rigidity)s*%(z)s))" % locals()
    def get_flux(self):
        return [[7860., 3550., 2200., 1430., 2120.]]
    def get_gamma(self):
        return [[2.66, 2.58, 2.63, 2.67, 2.63]]
    def get_rigidity(self):
        return [4*PeV]
    def __init__(self):
        flux = [self.build_lookup(zip(self.ptypes, f)) for f in self.get_flux()]
        gamma = [self.build_lookup(zip(self.ptypes, g)) for g in self.get_gamma()]
        rigidity = self.get_rigidity()
        CompiledFlux.__init__(self, "+".join([self.get_expression(f, g, r) for f, g, r in zip(flux, gamma, rigidity)]))

class GaisserH3a(GaisserHillas):  
    def get_flux(self):
        return super(GaisserH3a, self).get_flux() + [[20]*2 + [13.4]*3, [1.7]*2 + [1.14]*3]
    def get_gamma(self):
        return super(GaisserH3a, self).get_gamma() + [[2.4]*5, [2.4]*5]
    def get_rigidity(self):
        return super(GaisserH3a, self).get_rigidity() + [30*PeV, 2e3*PeV]

class GaisserH4a(GaisserH3a):
    def get_flux(self):
        return super(GaisserH4a, self).get_flux()[:-1] + [[200]]
    def get_gamma(self):
        return super(GaisserH4a, self).get_gamma()[:-1] + [[2.6]]
    def get_rigidity(self):
        return super(GaisserH4a, self).get_rigidity()[:-1] + [60e3*PeV]


In [3]:
data_type = 'corsika'
folder = '00000_00999' 
files = '*'
n_files_in_folder = 1938+99998+27727

#name = '_'+str(folder)+'_'+str(files)
#name2 = '_'+str(folder)
name = "trial"
name2 = "trial"

print(name, name2)

N_PRIM_CHILDREN = 3 
STRINGS_TO_SAVE = 10
N_Y_BINS = 60
N_X_BINS = 500
N_CHANNELS = 3
outer_strings = set([1,2,3,4,5,6,7,13,14,21,22,30,31,40,41,50,51,59,60,67,68,72,73,74,75,76,77,78])

trial trial


In [4]:
def get_rates_corsika(cwm,nfiles):
    flux = GaisserH4a()
    pflux = flux (cwm["PrimaryEnergy"],cwm["PrimaryType"])
    energy_integral = (cwm['EnergyPrimaryMax']**(cwm["PrimarySpectralIndex"]+1)-cwm['EnergyPrimaryMin']**(cwm["PrimarySpectralIndex"]+1))/(cwm["PrimarySpectralIndex"]+1)
    energy_weight = cwm['PrimaryEnergy']**cwm["PrimarySpectralIndex"]
    secs_per_year = 31536000
    w = secs_per_year*pflux *energy_integral/energy_weight * cwm["AreaSum"] / (cwm['NEvents'] * nfiles)
    return w

In [24]:
id_dtype = np.dtype(
    [
        ("run_id", np.uint32),
        ("sub_run_id", np.uint32),
        ("event_id", np.uint32),
        ("sub_event_id", np.uint32),
    ]
)
preds_dtype = np.dtype(
    [     
        ('n1', np.float32),
        ('n2', np.float32),
        ('n3', np.float32),
        ('n4', np.float32)
    ]
)
st_info_dtype = np.dtype(
    [
        ('q', np.float32),
        ('num', np.uint32),
        ('dist', np.float32)
    ]
)
map_dtype = np.dtype(
    [
        ("id", id_dtype),
        ('raw', np.int32),
        ('st_raw', np.int32,(3)),
        ('pulses', np.int32),
        ('st_pulses', np.int32,(3)),
        ('cal', np.int32),
        ('st_cal', np.int32,(3)),
        ('hlc', np.int32),
        ('st_hlc', np.int32,(3)),
        ('slc', np.int32),
        ('st_slc', np.int32,(3))
    ]
)

particle_dtype = np.dtype(
    [
        ("tree_id", np.uint32,(2)),
        ("pdg", np.int32),
        ("energy", np.float32),
        ("position", np.float32,(3)),
        ("direction", np.float32,(2)),
        ("time", np.float32),
        ("length", np.float32)
    ]
)
veto_dtype = np.dtype(                                             
    [                                                                             
        ("SPE_rlogl", np.float32),                                                      
        ("Cascade_rlogl", np.float32),
        ("SPE_rlogl_noDC", np.float32),                                                   
        ("Cascade_rlogl_noDC", np.float32),                                              
        ("FirstHitZ", np.float32),
        ("VHESelfVetoVertexPosZ", np.float32),                                             
        ("LeastDistanceToPolygon_Veto", np.float32)
    ]
)

hese_dtype = np.dtype(                                             
    [                                                                             
        ("qtot", np.float32),
        ("vheselfveto", np.bool_),
        ("vheselfvetovertexpos", np.float32,(3)),
        ("vheselfvetovertextime", np.float32),
        ("llhratio", np.float32)
    ]
)
CWEIGHT_KEY = "CorsikaWeightMap"
weight_dtype = np.dtype(
        [
            ("AreaSum" ,np.float32),
            ("Atmosphere",np.float32),
            ("CylinderLength",np.float32),
            ("CylinderRadius" ,np.float32),
            ("DiplopiaWeight",np.float32),
            ("EnergyPrimaryMax",np.float32),
            ("EnergyPrimaryMin",np.float32),
            ("FluxSum",np.float32),
            ("Multiplicity",np.float32),
            ("NEvents",np.float32),
            ("OldWeight",np.float32),
            ("OverSampling",np.float32),
            ("Polygonato",np.float32),
            ("PrimaryEnergy",np.float32),
            ("PrimarySpectralIndex",np.float32),
            ("PrimaryType",np.float32),
            ("ThetaMax",np.float32),
            ("ThetaMin" ,np.float32),
            ("TimeScale",np.float32),
            ("Weight",np.float32)
]                            
    )    
info_dtype = np.dtype(
    [
        ("id", id_dtype),
        ("image", np.float32, (N_X_BINS, N_Y_BINS, N_CHANNELS)),
        ("qtot", np.float32),
        ("cog", np.float32,(3)),
        ("moi", np.float32),
        ("ti", np.float32,(4)),
        ("qst", st_info_dtype, N_CHANNELS),
        ("qst_all", st_info_dtype, STRINGS_TO_SAVE),
        ("map", map_dtype),
        ("primary", particle_dtype),
        ("prim_daughter", particle_dtype),
        ("logan_veto", veto_dtype),
        ("hese_old", hese_dtype),
        ("hese", hese_dtype),
        ("llhcut",np.float32),
        ("weight", weight_dtype),
        ("weight_val",np.float32)

    ]
)
save_dtype = np.dtype(
    [
        ("id", id_dtype),
        ("preds", preds_dtype),
        ("dom", np.uint32),
        ("im_sum", np.float32,(3)),
        ("weight_val", np.float32),
        ("qtot", np.float32),
        ("cog", np.float32,(3)),
        ("moi", np.float32),
        ("ti", np.float32,(4)),
        ("map", map_dtype),
        ("qst", st_info_dtype, N_CHANNELS),
        ("qst_all", st_info_dtype, STRINGS_TO_SAVE),
        ("primary", particle_dtype),
        ("prim_daughter", particle_dtype),
        ("primary_child_energy", np.float32,(N_PRIM_CHILDREN)),
        ("primary_child_pdg", np.float32,(N_PRIM_CHILDREN)),
        ("logan_veto", veto_dtype),                                                  
        ("hese_old", hese_dtype),                                                  
        ("hese", hese_dtype),
        ("llhcut",np.float32),
        ("weight", weight_dtype),
        ("weight_it", np.float32)
    ]
)
save_2_dtype = np.dtype(
    [
        ("id", id_dtype),
        ("dom", np.uint32),
        ("im_sum", np.float32,(3)),
        ("out_st", np.uint32,(3)),
        ("preds", preds_dtype),
        ("weight_val", np.float32),
        ("qtot", np.float32),
        ("qst", st_info_dtype, N_CHANNELS),
        ("qst_all", st_info_dtype, STRINGS_TO_SAVE),
        ("primary", particle_dtype),
        ("prim_daughter", particle_dtype),
        ("primary_child_energy", np.float32,(N_PRIM_CHILDREN)),
        ("primary_child_pdg", np.float32,(N_PRIM_CHILDREN)),
        ("logan_veto", veto_dtype),                                                  
        ("hese_old", hese_dtype),                                                  
        ("hese", hese_dtype),                                                                                                    
        ("weight", weight_dtype)
    ]
)

In [7]:
import tensorflow as tf
import os

# Set which GPU to use.  This probably needs to be done before any other CUDA vars get defined.
# Use the command "nvidia-smi" to get association of a particular GPU with a particular number.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1,2,3,4"
from tensorflow.keras.models import load_model
model = load_model('/home/dup193/work/double_pulse/AC922/vgg16_200k_Qst_2000_2/vgg16_200k_QSt2000_dataset_norm_2.h5')
model_m = load_model('/home/dup193/work/double_pulse/AC922/vgg16_200k_Qst_2000_muon/vgg16_200k_QSt2000_dataset_norm_muon.h5')
model_3 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_300k_noQst2000_TvsE/vgg16_300k_noQSt2000_dataset_norm_TvsE_2.h5')
model_4 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_700k_Qst_2000_MuvsTau_3/vgg16_700k_QSt2000_dataset_norm_MuVsTau_3.h5')
#model_4 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_600k_Qst_2000_3class_classweight/vgg16_600k_QSt2000_dataset_norm_3class_classweight5.h5')

mean = 0.0012322452384978533 
std = 0.009694634936749935
mean_3 = 0.0002464977151248604
std_3 = 0.00594472698867321
mean_4 = 0.00036459346301853657
std_4 = 0.007035365793853998


In [25]:
size = 0
f1 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11057/*')
f2 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11058/*')
f3 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_10670/*')


files_grabbed = f1+f2+f3
print(len(files_grabbed))
for file_name in files_grabbed[:1]:
    x = np.load(file_name, mmap_mode="r")['arr_0']
    print(file_name, len(x))
    print(x['weight_val'])
    size = size +len(x)
print(size)

27107
/home/dup193/work/double_pulse/data/images_3str/Corsika_11057/Images_Corsika_11057_11000_11999_704_data.npz 2
[[nan]
 [nan]]
2


In [19]:
import time
pos = 0
print(name)
start = time.time()
data = np.lib.format.open_memmap('/fastio2/dasha/double_pulse/data_Corsika_11057_11058_10670_'+name2+'.npy', mode = 'w+', dtype =save_dtype, shape=(size,))
#n_files= len(glob.glob('/home/dup193/work/double_pulse/data/images_3str/Images_'+name+'*'))-1
#print(data.dtype)
f1 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11057/*')
f2 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11058/*')
f3 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_10670/*')
files_grabbed = f1+f2+f3
for file_name in files_grabbed:
    x = np.load(file_name, mmap_mode="r")['arr_0']
    #print(file_name, x.shape)
    y = np.zeros(x.shape[0],dtype = save_dtype)    
    im = (x['image']-mean)/std
    pred_n1 = model.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    pred_n2 = model_m.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    im = (x['image']-mean_3)/std_3
    pred_n3 = model_3.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    im = (x['image']-mean_4)/std_4
    pred_n4 = model_4.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
   
    for n,e in enumerate(x): 
        sums =[np.sum(e['image'][0,:,i,0]) for i in range(60)]
        max_dom = np.argmax(sums)
        im_sum = [-1,-1,-1]
        im_sum[0] = np.sum(e['image'][0,:,:,0])
        im_sum[1] = np.sum(e['image'][0,:,:,1])
        im_sum[2] = np.sum(e['image'][0,:,:,2])
        #print( e['qst']['num'][0])
        top_st = int(e['qst']['num'][0][0] in outer_strings)
        top3_st = sum([1 for i in e['qst']['num'][0] if i in outer_strings])
        top10_st = sum([1 for i in e['qst_all']['num'][0] if i in outer_strings])
        #print(top_st,top3_st,top10_st)
        preds = np.zeros(1,dtype = preds_dtype)    
        preds[['n1','n2','n3','n4']] = (pred_n1[n],pred_n2[n],pred_n3[n],pred_n4[n])
        weight_val = get_rates_corsika(e['weight'], n_files_in_folder)
        y[["id","dom","im_sum","preds","weight_val","qtot","qst","qst_all","moi","ti","map","primary",\
           "llhcut","prim_daughter","logan_veto","hese_old","hese","weight","weight_it"]][n]\
        =(e['id'], max_dom, im_sum, preds, weight_val,e['qtot'],e['qst'],e['qst_all'],e["moi"],e["ti"],e["map"],e['primary'],\
          e["llhcut"],e['prim_daughter'],e['logan_veto'],e['hese_old'],e['hese'],e['weight'],e['weight_val'])
    #print("\rPercent = "+str(round(n/x.shape[0]*100,3))+" "+str(n)+" of "+str(x.shape[0])+\
    #' Total = '+str(round((pos+n)/size*100,3))+" "+str(pos+n)+" of "+str(size), end="")
    data[pos:pos+len(x)] = y
    pos = pos + len(x)

end = time.time()
print(end - start)    


trial
5260.331630706787


In [ ]:
  ("moi", np.float32),
        ("ti", np.float32,(4)),
        ("map", map_dtype),
        ("qst", st_info_dtype, N_CHANNELS),
        ("qst_all", st_info_dtype, STRINGS_TO_SAVE),
        ("primary", particle_dtype),
        ("prim_daughter", particle_dtype),
        ("primary_child_energy", np.float32,(N_PRIM_CHILDREN)),
        ("primary_child_pdg", np.float32,(N_PRIM_CHILDREN)),
        ("logan_veto", veto_dtype),                                                  
        ("hese_old", hese_dtype),                                                  
        ("hese", hese_dtype),
        ("llhcut",np.float32),

In [ ]:
data0= np.load('/fastio2/dasha/double_pulse/data_'+name+'.npy', mmap_mode='r')
print(data0.shape, data0[0])

In [ ]:
data0['weight_val']

In [69]:
import tensorflow as tf
import os

# Set which GPU to use.  This probably needs to be done before any other CUDA vars get defined.
# Use the command "nvidia-smi" to get association of a particular GPU with a particular number.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3,4"
from tensorflow.keras.models import load_model
model = load_model('/home/dup193/work/double_pulse/vgg16_600k_QSt2000_dataset_norm_muon_cut_3class_trial.h5')

mean = 0.0012322452384978533 
std = 0.009694634936749935
for file_name in glob.glob('/home/dup193/work/double_pulse/data/images_3str/Images_NuE_3_1_1_data.npz'):
    x = np.load(file_name, mmap_mode="r")['arr_0']
    for n,e in enumerate(x):    #mean = np.mean(im)
        im = (e['image']-mean)/std
        pred = model.predict([im[:,:,:,:1],im[:,:,:,1:2],im[:,:,:,2:3]])
       # print(pred) 

[[0.351743   0.3420344  0.30622256]]
[[0.3499261  0.34121308 0.3088608 ]]
[[0.3469013  0.34506696 0.3080318 ]]
[[0.34924856 0.34063163 0.3101198 ]]
[[0.34952012 0.34130535 0.30917454]]
[[0.34613854 0.35066697 0.30319452]]
[[0.3490724  0.3405158  0.31041184]]
[[0.34750277 0.34415463 0.30834258]]
[[0.3492415  0.34442854 0.30633   ]]
[[0.34792626 0.34486374 0.30721   ]]
[[0.34873262 0.34358677 0.3076807 ]]
[[0.34876484 0.34038156 0.31085354]]
[[0.3501618  0.34255344 0.30728477]]
[[0.3498811  0.34286729 0.3072517 ]]
[[0.34895664 0.34015217 0.31089115]]
[[0.35010406 0.34224293 0.307653  ]]
[[0.3493635  0.34077737 0.30985904]]
[[0.34864643 0.3396816  0.311672  ]]
[[0.350181   0.34206718 0.3077518 ]]
[[0.3504231  0.34219095 0.30738598]]
[[0.34977847 0.34327966 0.30694184]]
[[0.3488504  0.34045097 0.3106986 ]]
[[0.3458053  0.34868678 0.305508  ]]
[[0.34869817 0.33981413 0.31148773]]
[[0.34865186 0.33971837 0.31162977]]
[[0.34931362 0.3432888  0.30739754]]
[[0.34914362 0.34024104 0.3106153 ]]
[